## Langevin dynamics in two dimensions
This notebook solves the two dimensional Langevin equation using a time-splitting integration method from [this paper](https://aip.scitation.org/doi/full/10.1063/1.4802990). The particle density, fluid viscosity, temperature, potential, and the simulation duration can be varied interactively. Particle size is fixed at 2 $\mu m$ diameter and the time step is fixed at 0.5 ms. 

In [55]:
# Import useful packages
import numpy as np
import time
from collections import deque
import ipywidgets as widgets
from IPython.display import display
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure 
output_notebook()

Loading BokehJS ...

In [56]:
# Defines a class for the Langevin simulation
class Langevin2D:
    
    # Initializing variables
    def __init__(self, mass_density_of_particle, fluid_viscosity, 
                 temperature, strength_of_potential):
        self.partrad = 1e-6
        self.dt = 0.5e-3
        self.boxsize = 50e-6
        self.center = self.boxsize / 2
        self.partden = mass_density_of_particle
        self.flvisc = fluid_viscosity
        self.temp = temperature
        self.pot = strength_of_potential * 1e-8
        self.reset()
        self.calc_params()
        
    # Reseting positions and velocities to zero
    def reset(self):
        self.x = np.zeros(2)
        self.p = np.zeros(2)
        self.pmid = np.zeros(2)
        pos_x.clear()
        pos_y.clear()
        
    # Calculating useful parameters used in solving the Langevin equation
    def calc_params(self):
        self.partmass = self.partden * 1.333 * np.pi * self.partrad**3
        self.stokes = 6.0 * np.pi * self.flvisc * self.partrad
        self.gamma = self.stokes / self.partmass
        self.c1 = np.exp(-self.gamma * self.dt)
        self.c2 = np.sqrt(kb * self.temp * (1 - self.c1**2))
        
    # Run the Langevin update
    def run(self, r1, r2, target):
        global pos_x, pos_y
        pos_x.append(self.x[0] * 1e6)
        pos_y.append(self.x[1] * 1e6)
        self.pmid = self.p - self.dt * self.pot * self.x / 2
        self.x = self.x + self.dt * self.pmid / 2 / self.partmass
        self.pmid = self.c1 * self.pmid + self.c2 * np.random.randn(2) * np.sqrt(self.partmass)
        self.x = self.x + self.dt * self.pmid / 2 / self.partmass
        self.p = self.pmid - self.dt * self.pot * self.x / 2
        self.check_boundary()
        self.plot_trajectories(r1, r2, target)
    
    # Apply periodic boundary conditions 
    def check_boundary(self):
        # Applying periodic boundaries
        for i in range(2):
            if self.x[i] <  -self.boxsize * 0.5: 
                self.x[i] = self.x[i] + self.boxsize
            if self.x[i] >=  self.boxsize * 0.5:
                self.x[i] = self.x[i] - self.boxsize
    
    # Update arrays for plotting
    def plot_trajectories(self, r1, r2, target):
        r1.data_source.data['x'] = pos_x
        r1.data_source.data['y'] = pos_y
        r2.data_source.data['x'] = [self.x[0] * 1e6]
        r2.data_source.data['y'] = [self.x[1] * 1e6]
        push_notebook(handle=target)

In [57]:
kb = 1.380e-23  # Boltzmann constant
# Queue for holding recent particle positions
pos_x = deque('', 100)
pos_y = deque('', 100)

# Start the Langevin simulation
def start_Langevin_sim(mass_density_of_particle=1000.0, 
                       fluid_viscosity=9e-4, temperature=300, strength_of_potential=1.0,
                       simulation_duration=0.5):
    pos_x.clear()
    pos_y.clear()
    # Create langevin class object
    l2d = Langevin2D(mass_density_of_particle, fluid_viscosity, temperature, strength_of_potential)
    # Initialize figure for plotting trajectories
    p1 = figure(plot_width=500, plot_height=400, tools='',
                x_range=(-l2d.boxsize*1e6/2, l2d.boxsize*1e6/2), 
                y_range=(-l2d.boxsize*1e6/2, l2d.boxsize*1e6/2))
    p1.xaxis.axis_label = 'x (um)'
    p1.yaxis.axis_label = 'y (um)'
    p1.toolbar.logo = None
    runtime = simulation_duration * 0.1 
    r1 = p1.circle(pos_x, pos_y, size=1, color='chocolate', alpha=1)
    r2 = p1.circle([l2d.x[0] * 1e6], [l2d.x[1] * 1e6], 
                   radius=int(l2d.partrad*1e6), alpha=0.5, color='maroon')
    target = show(p1, notebook_handle=True)
    # Langevin time-steping loop
    for t in range(int(runtime/l2d.dt)):
        l2d.run(r1, r2, target)
        time.sleep(0.05)

In [58]:
# Interactive controls
style = {'description_width': 'initial'}    
mass_density_of_particle = widgets.IntSlider(description='Particle mass density', 
                                              style=style, min=500, max=5000, 
                                              step=100, value=1000, continuous_update=False)
fluid_viscosity = widgets.FloatLogSlider(description='Dynamic viscosity', 
                                         style=style, base=10, min=-6, max=-1, 
                                         step=-4, value=9e-4, continuous_update=False)
temperature = widgets.IntSlider(description='Temperature (K)', 
                                style=style, min=50, max=600, 
                                step=50, value=300, continuous_update=False)
strength_of_potential = widgets.IntSlider(description='Potential',
                                            style=style, min=0, max=4, 
                                            step=1, value=2, continuous_update=False)
simulation_duration = widgets.ToggleButtons(options=[('Short', 0.5), ('Regular', 2), ('Long', 5)],
                                            description='Simulation duration', disabled=False, style=style)

# Creating the interactive controls ui
widget_ui = widgets.VBox([widgets.HBox([mass_density_of_particle, fluid_viscosity]), 
                         widgets.HBox([temperature, strength_of_potential]), simulation_duration])
widget_out = widgets.interactive_output(start_Langevin_sim, 
                                        {'mass_density_of_particle': mass_density_of_particle,
                                         'fluid_viscosity': fluid_viscosity,
                                         'temperature': temperature,
                                         'strength_of_potential': strength_of_potential,
                                         'simulation_duration': simulation_duration})

In [59]:
# Display the controls and output
display(widget_ui, widget_out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/html': '\n\n\n\n\n\n  <div class="bk-root" id="…